In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from bs4 import BeautifulSoup as bs
import requests
import regex as re
import unicodedata

#Downloading the initial json file
import time
import hashlib
import json
import urllib.request
from urllib.request import urlopen, Request

In [2]:
url1 = 'https://www.fda.gov/datatables/views/ajax?search_api_fulltext=&search_api_fulltext_issuing_office=&field_letter_issue_datetime=All&field_change_date_closeout_letter=&field_change_date_response_letter=&field_change_date_2=All&field_letter_issue_datetime_2=&draw=2&columns[0][data]=0&columns[0][name]=&columns[0][searchable]=true&columns[0][orderable]=true&columns[0][search][value]=&columns[0][search][regex]=false&columns[1][data]=1&columns[1][name]=&columns[1][searchable]=true&columns[1][orderable]=true&columns[1][search][value]=&columns[1][search][regex]=false&columns[2][data]=2&columns[2][name]=&columns[2][searchable]=true&columns[2][orderable]=true&columns[2][search][value]=&columns[2][search][regex]=false&columns[3][data]=3&columns[3][name]=&columns[3][searchable]=true&columns[3][orderable]=true&columns[3][search][value]=&columns[3][search][regex]=false&columns[4][data]=4&columns[4][name]=&columns[4][searchable]=true&columns[4][orderable]=true&columns[4][search][value]=&columns[4][search][regex]=false&columns[5][data]=5&columns[5][name]=&columns[5][searchable]=true&columns[5][orderable]=true&columns[5][search][value]=&columns[5][search][regex]=false&columns[6][data]=6&columns[6][name]=&columns[6][searchable]=true&columns[6][orderable]=true&columns[6][search][value]=&columns[6][search][regex]=false&columns[7][data]=7&columns[7][name]=&columns[7][searchable]=true&columns[7][orderable]=false&columns[7][search][value]=&columns[7][search][regex]=false&start=0&length=4000&search[value]=&search[regex]=false&_drupal_ajax=1&_wrapper_format=drupal_ajax&pager_element=0&view_args=&view_base_path=inspections-compliance-enforcement-and-criminal-investigations/compliance-actions-and-activities/warning-letters/datatables-data&view_display_id=warning_letter_solr_block&view_dom_id=1b5f02673e058c411472545b66423c0c07c1d6a20e44e1d9ff445533d62fca96&view_name=warning_letter_solr_index&view_path=/inspections-compliance-enforcement-and-criminal-investigations/compliance-actions-and-activities/warning-letters&total_items=2894&_=1648381464969'
response = urlopen(url1).read()


json_path = url1
json_path1 = json_path.replace("'", "")
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".json", 'wb')
    file.write(response.read())
    file.close()
 
download_file(json_path1, "warning_letter")

In [3]:
df = pd.read_json("./warning_letter.json")

In [4]:
df =  df["data"]

In [5]:
df = pd.DataFrame(dict(zip(df.index, df.values))).T

In [6]:
df.head()

,0,1,2,3,4,5,6,7
0,"<time datetime=""2022-03-24T19:00:00Z"">03/24/20...","<time datetime=""2022-03-24T04:00:00Z"">03/24/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,,,
1,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-02-03T05:00:00Z"">02/03/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Drug Evaluation and Research | CDER,Failure to Register and List,,"<a href=""/inspections-compliance-enforcement-a...",
2,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-03-15T04:00:00Z"">03/15/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,,,
3,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-03-18T04:00:00Z"">03/18/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,,,
4,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-02-23T05:00:00Z"">02/23/20...","<a href=""/inspections-compliance-enforcement-a...",Office of Human and Animal Food Operations- We...,"CGMP/Food for Animals/Prepared, Packed or Held...",,,


In [7]:
cols = ["posted_date", "letter_issue_date", "company_name_and_link", "issuing_office",\
        "subject","response_letter_date_and_link","closeout_letter_date_and_link", "check"]

df.columns = cols

In [8]:
# Replacing blanks in response_letter and closeout_letter with NAN Values in the df
df["response_letter_date_and_link"][df["response_letter_date_and_link"].str.contains("href") == False] = np.NaN
df["closeout_letter_date_and_link"][df["closeout_letter_date_and_link"].str.contains("href") == False] = np.NaN

In [9]:
df["check"].str.contains('\w', regex=True).sum()

0

In [10]:
df.drop(["check"], axis=1, inplace = True)

In [11]:
df.head()

,posted_date,letter_issue_date,company_name_and_link,issuing_office,subject,response_letter_date_and_link,closeout_letter_date_and_link
0,"<time datetime=""2022-03-24T19:00:00Z"">03/24/20...","<time datetime=""2022-03-24T04:00:00Z"">03/24/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,NaN,NaN
1,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-02-03T05:00:00Z"">02/03/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Drug Evaluation and Research | CDER,Failure to Register and List,NaN,"<a href=""/inspections-compliance-enforcement-a..."
2,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-03-15T04:00:00Z"">03/15/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,NaN,NaN
3,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-03-18T04:00:00Z"">03/18/20...","<a href=""/inspections-compliance-enforcement-a...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,NaN,NaN
4,"<time datetime=""2022-03-22T13:20:00Z"">03/22/20...","<time datetime=""2022-02-23T05:00:00Z"">02/23/20...","<a href=""/inspections-compliance-enforcement-a...",Office of Human and Animal Food Operations- We...,"CGMP/Food for Animals/Prepared, Packed or Held...",NaN,NaN


In [12]:
def date_extract(x):
    soup = bs(x, "lxml")
    date = soup.find("time").text.strip()
    return date

In [13]:
def company_name_and_link(x):
    soup = bs(x, "lxml")
    company_name = soup.find("a").text.strip()
    link = "https://www.fda.gov" + soup.find("a")["href"].strip()
    return company_name, link

In [14]:
df[["posted_date", "letter_issue_date"]] = df[["posted_date", "letter_issue_date"]].applymap(lambda x: date_extract(x))

In [15]:
df["company_name_and_link"] = df["company_name_and_link"].apply(company_name_and_link)

In [16]:
df['company_name'], df['warning_letter_link'] = zip(*df["company_name_and_link"])

In [17]:
df[["issuing_office", "subject"]] = df[["issuing_office", "subject"]].applymap(lambda x: x.strip())

In [18]:
df[df["response_letter_date_and_link"].notnull()]

,posted_date,letter_issue_date,company_name_and_link,issuing_office,subject,response_letter_date_and_link,closeout_letter_date_and_link,company_name,warning_letter_link
200,12/07/2021,11/09/2021,"(Synaptent, LLC, https://www.fda.gov/inspectio...",Office of Human and Animal Food Operations-East 6,New Drug/Misbranded,"<a href=""/inspections-compliance-enforcement-a...",NaN,"Synaptent, LLC",https://www.fda.gov/inspections-compliance-enf...
928,09/29/2020,09/09/2020,"(Foothills Professional Pharmacy, LTD, https:/...",Division of Pharmaceutical Quality Operations IV,Compounding Pharmacy/Adulterated Drug Products,"<a href=""/inspections-compliance-enforcement-a...",NaN,"Foothills Professional Pharmacy, LTD",https://www.fda.gov/inspections-compliance-enf...
1635,09/03/2019,08/26/2019,"(Absonutrix, LLC, https://www.fda.gov/inspecti...",Division of Human and Animal Food Operations E...,Dietary Supplement/Adulterated<br />,"<a href=""/inspections-compliance-enforcement-a...",NaN,"Absonutrix, LLC",https://www.fda.gov/inspections-compliance-enf...
1655,08/20/2019,07/22/2019,"(Almark Foods Inc, https://www.fda.gov/inspect...",Division of Human and Animal Food Operations E...,Food/Prepared Packed or Held Under Insanitary ...,"<a href=""/inspections-compliance-enforcement-a...",NaN,Almark Foods Inc,https://www.fda.gov/inspections-compliance-enf...
1994,10/30/2018,10/18/2018,(Custom RX LLC dba Custom Rx Pharmacy and Well...,Detroit District Office,Custom RX LLC dba Custom Rx Pharmacy and Welln...,"<a href=""/inspections-compliance-enforcement-a...","<a href=""/inspections-compliance-enforcement-a...",Custom RX LLC dba Custom Rx Pharmacy and Welln...,https://www.fda.gov/inspections-compliance-enf...
2126,07/24/2018,07/19/2018,"(Niche Pharmaceuticals, Inc, https://www.fda.g...",Dallas District Office,Unapproved New Drugs/Misbranded,"<a href=""/inspections-compliance-enforcement-a...","<a href=""/inspections-compliance-enforcement-a...","Niche Pharmaceuticals, Inc",https://www.fda.gov/inspections-compliance-enf...


In [19]:
def letter_date_and_link(x):
    soup = bs(x, "lxml")
    date = soup.find("a").text.strip()
    link = "https://www.fda.gov" + soup.find("a")["href"].strip()
    return date, link

Getting Response Letter Date and Link

In [20]:
get_rows_response_letter = df["response_letter_date_and_link"].notnull()

In [21]:
df["response_letter_date_and_link"][get_rows_response_letter] = df["response_letter_date_and_link"][get_rows_response_letter].\
apply(letter_date_and_link)

In [22]:
df[['response_date', 'response_link']] = df[get_rows_response_letter].apply(lambda x:x.response_letter_date_and_link,\
                                                            result_type='expand', axis=1)

Getting Closeout Letter date and Links

In [23]:
get_rows_closeout_letter = df["closeout_letter_date_and_link"].notnull()

In [24]:
df["closeout_letter_date_and_link"][get_rows_closeout_letter] = df["closeout_letter_date_and_link"][get_rows_closeout_letter].\
apply(letter_date_and_link)

In [25]:
df[['closeout_date', 'closeout_link']] = df[get_rows_closeout_letter].apply(lambda x:x.closeout_letter_date_and_link,\
                                                            result_type='expand', axis=1)

In [26]:
df.head()

,posted_date,letter_issue_date,company_name_and_link,issuing_office,subject,response_letter_date_and_link,closeout_letter_date_and_link,company_name,warning_letter_link,response_date,response_link,closeout_date,closeout_link
0,03/24/2022,03/24/2022,"(Ohm Men Vape Shop LLC, https://www.fda.gov/in...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,NaN,NaN,Ohm Men Vape Shop LLC,https://www.fda.gov/inspections-compliance-enf...,NaN,NaN,NaN,NaN
1,03/22/2022,02/03/2022,"(ECI Pharmaceuticals, LLC, https://www.fda.gov...",Center for Drug Evaluation and Research | CDER,Failure to Register and List,NaN,"(03/22/2022, https://www.fda.gov/inspections-c...","ECI Pharmaceuticals, LLC",https://www.fda.gov/inspections-compliance-enf...,NaN,NaN,03/22/2022,https://www.fda.gov/inspections-compliance-enf...
2,03/22/2022,03/15/2022,"(Steam Puff Vapor LLC DBA Vapery 2, https://ww...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,NaN,NaN,Steam Puff Vapor LLC DBA Vapery 2,https://www.fda.gov/inspections-compliance-enf...,NaN,NaN,NaN,NaN
3,03/22/2022,03/18/2022,"(Marlin Steam Co, https://www.fda.gov/inspecti...",Center for Tobacco Products,Family Smoking Prevention and Tobacco Control ...,NaN,NaN,Marlin Steam Co,https://www.fda.gov/inspections-compliance-enf...,NaN,NaN,NaN,NaN
4,03/22/2022,02/23/2022,"(OC Raw Dog LLC, https://www.fda.gov/inspectio...",Office of Human and Animal Food Operations- We...,"CGMP/Food for Animals/Prepared, Packed or Held...",NaN,NaN,OC Raw Dog LLC,https://www.fda.gov/inspections-compliance-enf...,NaN,NaN,NaN,NaN


Creating a Scrapper to scrape through links and extract text

In [27]:
def div_scrapper(x):
    
    scrapped_text = ""
    r = requests.get(x)
    soup = bs(r.text, "lxml")
    find_div = soup.find("div", class_ = "col-md-8 col-md-push-2")
    scrapped_text = re.sub(r'\n+', '\n', unicodedata.normalize("NFKD", find_div.text))
    scrapped_text = scrapped_text.strip()
    
    return scrapped_text

In [28]:
df["warning_letter_text"] = df["warning_letter_link"].apply(div_scrapper)
df['response_text'] = df[get_rows_response_letter]["response_link"].apply(div_scrapper)
df['closeout_text'] = df[get_rows_closeout_letter]["closeout_link"].apply(div_scrapper)
df.to_csv("warning_letters.csv")